In [3]:
import pyrealsense2 as rs
import json
import time

def get_color_sensor(profile):
    dev = profile.get_device()
    for s in dev.sensors:
        name = s.get_info(rs.camera_info.name).lower()
        if "color" in name or "rgb" in name:
            return s
    return None

def apply_preset(profile, preset):
    """
    profile: pipeline profile returned by pipeline.start(cfg)
    preset: dict with keys possibly 'exposure','white_balance','gain'
    """
    sensor = get_color_sensor(profile)
    if sensor is None:
        raise RuntimeError("Color sensor not found")

    # Disable autos if supported
    try:
        if sensor.supports(rs.option.enable_auto_exposure):
            sensor.set_option(rs.option.enable_auto_exposure, 0)
    except Exception as e:
        print("Warning: could not disable auto exposure:", e)

    try:
        if sensor.supports(rs.option.enable_auto_white_balance):
            sensor.set_option(rs.option.enable_auto_white_balance, 0)
    except Exception as e:
        print("Warning: could not disable auto white balance:", e)

    # Set saved values (if supported)
    try:
        if 'exposure' in preset and sensor.supports(rs.option.exposure):
            sensor.set_option(rs.option.exposure, float(preset['exposure']))
        if 'white_balance' in preset and sensor.supports(rs.option.white_balance):
            sensor.set_option(rs.option.white_balance, float(preset['white_balance']))
        if 'gain' in preset and sensor.supports(rs.option.gain):
            sensor.set_option(rs.option.gain, float(preset['gain']))
    except Exception as e:
        print("Warning: could not set some options:", e)

    # readback verify
    readback = {}
    try:
        if sensor.supports(rs.option.exposure):
            readback['exposure'] = sensor.get_option(rs.option.exposure)
        if sensor.supports(rs.option.white_balance):
            readback['white_balance'] = sensor.get_option(rs.option.white_balance)
        if sensor.supports(rs.option.gain):
            readback['gain'] = sensor.get_option(rs.option.gain)
    except Exception as e:
        print("Warning: could not read back options:", e)
    print("Applied preset, readback:", readback)
    return readback

# 사용 예시
pipeline = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, 1280,720, rs.format.bgr8, 30)
profile = pipeline.start(cfg)
time.sleep(0.1)  # 안전 마진

with open('rs_locked_preset.json','r') as f:
    preset = json.load(f)

apply_preset(profile, preset)
# 이후 캡처/캘리 작업 진행


Applied preset, readback: {'exposure': 166.0, 'white_balance': 4600.0, 'gain': 64.0}


{'exposure': 166.0, 'white_balance': 4600.0, 'gain': 64.0}

In [5]:
# capture_calib_images.py
import time, os, cv2, numpy as np
import pyrealsense2 as rs

SAVE_DIR = "calib_images"
os.makedirs(SAVE_DIR, exist_ok=True)


pipeline = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, 1280,720, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 1280,720, rs.format.z16, 30)
profile = pipeline.start(cfg)
apply_preset(profile, preset)

count = 0
print("Manual mode: press 's' to save, 'q' to quit.")
try:
    while True:
        frames = pipeline.wait_for_frames()
        color = np.asanyarray(frames.get_color_frame().get_data())
        display = cv2.resize(color, (int(color.shape[1]/2), int(color.shape[0]/2)))
        cv2.imshow("capture", display)
        k = cv2.waitKey(1) & 0xFF
        if k == ord('s'):
            fname = os.path.join(SAVE_DIR, f"calib_{count:03d}.png")
            cv2.imwrite(fname, color)
            print("Saved", fname)
            count += 1
        elif k == ord('q'):
            break
finally:
    cv2.destroyAllWindows()
    pipeline.stop()


Applied preset, readback: {'exposure': 166.0, 'white_balance': 4600.0, 'gain': 64.0}
Manual mode: press 's' to save, 'q' to quit.
